In [8]:
from IPython.display import Video
import cv2
import os
import random

In [16]:
def generate_frame(path, frame_number):
    #create folder if it doesn't exist
    if not os.path.exists('Private_dataset/' + path):
        os.makedirs('Private_dataset/' + path)
        

    vidcap = cv2.VideoCapture('Private_dataset/{}.MOV'.format(path))

    #sample 40 frames uniformly from the video
    frame_count = int(vidcap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_indices = sorted(random.sample(range(frame_count), frame_number))
    vidcap.set(cv2.CAP_PROP_POS_FRAMES, frame_indices[0])
    success, image = vidcap.read()
    count = 0
    for i in range(1, frame_count):
        if i == frame_indices[count]:
            #rotate the image 180 degrees
            image = cv2.rotate(image, cv2.ROTATE_180)
            cv2.imwrite(f"./Private_dataset/{format(path)}/{format(count, '04d')}.jpg", image)
            count += 1
            if count >= len(frame_indices):
                break
        success, image = vidcap.read()
        if not success:
            break


generate_frame('Xiang', 40)
generate_frame('Jiaxun', 40)
generate_frame('Bowen', 40)